In [1]:
import pandas as pd

# Load the data
df = pd.read_csv('/Users/ernestgaisie/Desktop/Final Projects/CANADA_MORTGAGE_RATES_ANALYSIS/cleaned_34100133.csv')  # Replace with the actual path

# Inspect the first few rows and column names
print(df.head())
print(df.columns)

   REF_DATE                                       GEO             DGUID  \
0      1987   Corner Brook, Newfoundland and Labrador      2011S0504015   
1      1987         Gander, Newfoundland and Labrador  2011A00051006009   
2      1987         Gander, Newfoundland and Labrador  2011A00051006009   
3      1987         Gander, Newfoundland and Labrador  2011A00051006009   
4      1987  Labrador City, Newfoundland and Labrador  2011A00051010032   

                            Type of structure         Type of unit      UOM  \
0  Apartment structures of six units and over    Two bedroom units  Dollars   
1  Apartment structures of six units and over    One bedroom units  Dollars   
2  Apartment structures of six units and over    Two bedroom units  Dollars   
3  Apartment structures of six units and over  Three bedroom units  Dollars   
4  Apartment structures of six units and over    One bedroom units  Dollars   

   UOM_ID SCALAR_FACTOR  SCALAR_ID    VECTOR COORDINATE  VALUE  STATUS  \


/var/folders/nq/174f2qcn6cl3hlkkbcr98vw00000gn/T/ipykernel_2081/2452860224.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/ernestgaisie/Desktop/Final Projects/CANADA_MORTGAGE_RATES_ANALYSIS/cleaned_34100133.csv')  # Replace with the actual path


In [2]:
# Set the target variable (mortgage price)
target = 'VALUE'

# Select relevant features
features = ['GEO', 'Type of structure', 'Type of unit']

In [3]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le_geo = LabelEncoder()
le_structure = LabelEncoder()
le_unit = LabelEncoder()

# Apply label encoding to each categorical feature
df['GEO_encoded'] = le_geo.fit_transform(df['GEO'])
df['Type of structure_encoded'] = le_structure.fit_transform(df['Type of structure'])
df['Type of unit_encoded'] = le_unit.fit_transform(df['Type of unit'])

In [6]:
# Create reverse mappings to decode the predictions later
geo_reverse_mapping = {v: k for k, v in dict(zip(le_geo.classes_, le_geo.transform(le_geo.classes_))).items()}
structure_reverse_mapping = {v: k for k, v in dict(zip(le_structure.classes_, le_structure.transform(le_structure.classes_))).items()}
unit_reverse_mapping = {v: k for k, v in dict(zip(le_unit.classes_, le_unit.transform(le_unit.classes_))).items()}

# Print the mappings for your reference
print("GEO Reverse Mapping:")
for key, value in geo_reverse_mapping.items():
    print(f"{key}: '{value}'")

print("\nType of Structure Reverse Mapping:")
for key, value in structure_reverse_mapping.items():
    print(f"{key}: '{value}'")

print("\nType of Unit Reverse Mapping:")
for key, value in unit_reverse_mapping.items():
    print(f"{key}: '{value}'")

GEO Reverse Mapping:
0: 'Abbotsford-Mission, British Columbia'
1: 'Alma, Quebec'
2: 'Amos, Quebec'
3: 'Asbestos, Quebec'
4: 'Baie-Comeau, Quebec'
5: 'Barrie, Ontario'
6: 'Bathurst, New Brunswick'
7: 'Bay Roberts, Newfoundland and Labrador'
8: 'Belleville, Ontario'
9: 'Bracebridge, Ontario'
10: 'Brandon, Manitoba'
11: 'Brantford, Ontario'
12: 'Brighton, Ontario'
13: 'Brock, Ontario'
14: 'Brockville, Ontario'
15: 'Brooks, Alberta'
16: 'Bécancour, Quebec'
17: 'Calgary, Alberta'
18: 'Campbell River, British Columbia'
19: 'Campbellton, New Brunswick part, New Brunswick/Quebec'
20: 'Camrose, Alberta'
21: 'Canmore, Alberta'
22: 'Cape Breton, Nova Scotia'
23: 'Centre Wellington, Ontario'
24: 'Charlottetown, Prince Edward Island'
25: 'Chatham-Kent, Ontario'
26: 'Chibougamau, Quebec'
27: 'Chilliwack, British Columbia'
28: 'Cobourg, Ontario'
29: 'Cold Lake, Alberta'
30: 'Collingwood, Ontario'
31: 'Corner Brook, Newfoundland and Labrador'
32: 'Cornwall, Ontario'
33: 'Courtenay, British Columbia'
3

In [7]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (y)
X = df[['GEO_encoded', 'Type of structure_encoded', 'Type of unit_encoded']]
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 43126.10566065108


In [9]:
# Example: Use these encoded values for prediction
geo_value = 50  # Encoded value for 'Gander, Newfoundland and Labrador'
structure_value = 0  # Encoded value for 'Apartment structures of six units and over'
unit_value = 3  # Encoded value for 'Two bedroom units'

# Create a new DataFrame with these encoded values
new_data = pd.DataFrame({
    'GEO_encoded': [geo_value],
    'Type of structure_encoded': [structure_value],
    'Type of unit_encoded': [unit_value]
})

# Predict the mortgage price for the new data
predicted_price = model.predict(new_data)
print(f'Predicted Mortgage Price: {predicted_price[0]}')

Predicted Mortgage Price: 557.347867942827


In [10]:
# Decode the prediction back to original categorical values
geo_original = geo_reverse_mapping[geo_value]
structure_original = structure_reverse_mapping[structure_value]
unit_original = unit_reverse_mapping[unit_value]

# Print the decoded prediction information
print(f'Predicted Mortgage Price for {geo_original}, {structure_original}, {unit_original}: {predicted_price[0]}')

Predicted Mortgage Price for Gander, Newfoundland and Labrador, Apartment structures of six units and over, Two bedroom units: 557.347867942827


In [20]:
# Example: Use these encoded values for prediction
geo_value = 217  # Encoded value for 'Gander, Newfoundland and Labrador'
structure_value = 0  # Encoded value for 'Apartment structures of six units and over'
unit_value = 3  # Encoded value for 'Two bedroom units'

# Create a new DataFrame with these encoded values
new_data = pd.DataFrame({
    'GEO_encoded': [geo_value],
    'Type of structure_encoded': [structure_value],
    'Type of unit_encoded': [unit_value]
})

# Predict the mortgage price for the new data
predicted_price = model.predict(new_data)
print(f'Predicted Mortgage Price: {predicted_price[0]}')

Predicted Mortgage Price: 1483.373184801988


In [21]:
# Decode the prediction back to original categorical values
geo_original = geo_reverse_mapping[geo_value]
structure_original = structure_reverse_mapping[structure_value]
unit_original = unit_reverse_mapping[unit_value]

# Print the decoded prediction information
print(f'Predicted Mortgage Price for {geo_original}, {structure_original}, {unit_original}: {predicted_price[0]}')

Predicted Mortgage Price for Yellowknife, Northwest Territories, Apartment structures of six units and over, Two bedroom units: 1483.373184801988
